In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook, tqdm
from scipy.spatial.distance import jaccard

from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVD, SVDpp, accuracy
from surprise.model_selection import KFold, train_test_split, cross_validate, GridSearchCV

import warnings
warnings.simplefilter('ignore')

In [2]:
# !find * -iname 'movies.c*' -or -iname 'ratings.csv' -print -or -iname 'Library' -prune -or -iname 'Dropbox' -prune
# !find * -iname 'movies.c*' -or -iname 'ratings.csv' -print -or -iname 'Library' -prune

In [3]:
movies = pd.read_csv('movies.csv') # Подгружаем данные
ratings = pd.read_csv('ratings.csv')

In [4]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True) # Объеденяем 'фильмы' и 'Оценки'
movies_with_ratings.dropna(inplace=True) # Удаляем пропуски
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [5]:
num_movies = movies_with_ratings.movieId.unique().shape[0] # len() Получаем колличество уникальных ID фильмов
uniques = movies_with_ratings.movieId.unique() # Список уникальных ID фильмов <class 'numpy.ndarray'>


user_vector = {} # Формируем словарь (векторов), где {key=ID_юзера: values=array([Рейтинга])}
for user, group in movies_with_ratings.groupby('userId'):
    user_vector[user] = np.zeros(num_movies)
    
    for i in range(len(group.movieId.values)):
        m = np.argwhere(uniques==group.movieId.values[i])[0][0]
        r = group.rating.values[i]
        user_vector[user][m] = r

        
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId, 
    'iid': movies_with_ratings.title, 
    'rating': movies_with_ratings.rating
}) # Формируем новый 'dataset' который будет учавствовать в нашей модели из библиотеки 'surprise'


dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [6]:
reader = Reader(rating_scale=(0.5, 5.0)) # Указываем рейтинг где 0.5 минимальный, а 5.0 максимальный
data = Dataset.load_from_df(dataset, reader) # Преобразовываем 'dataset' в необходимый формат библиотеки 'surprise'

trainset, testset = train_test_split(data, test_size=.15, random_state=42) # Делим на train и test выборку

algo = SVDpp(n_factors=20, n_epochs=20) # Наша модель SVD++ (https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp)
algo.fit(trainset) # Обучаем модель на 'train'

test_pred = algo.test(testset) # Проверяем на 'test'
accuracy.rmse(test_pred, verbose=True) # Смотрим на 'Среднюю Квадратическую Ошибку' (Root Mean Square Error)

# Root Mean Square Error (RMSE) is the standard deviation of the residuals (prediction errors). \
# Residuals are a measure of how far from the regression line data points are; \
# RMSE is a measure of how spread out these residuals are. \
# In other words, it tells you how concentrated the data is around the line of best fit. 

RMSE: 0.8565


0.8565172138111715

In [106]:
def recommendation(uid=2.0, neighbors=5, ratin=4.5, films=5, top=5):
    
    '''
    uid       - идентификационный номер пользователя, который запросил рекомендации
    neighbors - указываем необходимое количество похожих пользователей на 'uid' для поиска
    ratin     - рейтинг фильмов похожих пользователей на 'uid'
    films     - количество фильмов для предсказания оценки и сортировки
    top       - количество рекомендованных фильмов пользователю 'uid'
    '''
    
    titles = [key for key in user_vector.keys() if key != uid] # только те ключи где != ID чтобы не брать фильмы пользователя которые он посмотрел и оценил
    distances = [jaccard(user_vector[uid], user_vector[key]) for key in user_vector.keys() if key != uid] # Джаккард (https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jaccard.html#scipy.spatial.distance.jaccard)

    best_indexes = np.argsort(distances)[:neighbors] # Сортировка
    similar_users = np.array([(titles[i], distances[i]) for i in best_indexes])[:, 0]

    movies_with_ratings.sort_values('timestamp', inplace=True) # Сортировка по времени
    
    movies = np.array(list(set([]))) # Конструкция list(set()) для исключения дублей
    for user in similar_users:
        a = np.array(movies_with_ratings[movies_with_ratings.rating >= ratin][movies_with_ratings.userId == user][-films:].title)
        movies = np.concatenate([a, movies])

    user_movies = movies_with_ratings[movies_with_ratings.userId == uid].title.unique()

    scores = list(set([algo.predict(uid=uid, iid=movie).est for movie in movies]))
    titles_s = list(set([movie for movie in movies]))

    best_indexes = np.argsort(scores)[-top:] # Сортировка
    
    scores_r = [scores[i] for i in reversed(best_indexes)] #list(reversed([1, 2, 3, 4])) -> [4, 3, 2, 1]
    titles_r = [titles_s[i] for i in reversed(best_indexes)]
    
    # Объеденяем в один dataframe для вывода рекомендаций
    df1, df2 = pd.DataFrame(data=titles_r).reset_index(), pd.DataFrame(data=scores_r).reset_index()
    df1.columns, df2.columns = ['index','films'], ['index','scores']
    df = pd.merge(df1, df2, on='index')
    df['rank'] = df.scores.rank(ascending=False).astype('int')
    data = df[['rank', 'films', 'scores']]
    
    return data

In [133]:
''' Пользователь 2
    Похожих пользователей 10
    Из фильмов с минимальным рейтингом 4.5 у похожих пользователей
    По топ 10 фильмам похожих пользователей
    Топ рекомендаций 10 фильмов для пользователя '''

data = recommendation(uid=2.0, neighbors=10, ratin=4.5, films=10, top=10)

In [135]:
data.head(10)

,rank,films,scores
0,1,"Three Billboards Outside Ebbing, Missouri (2017)",4.297498
1,2,Fight Club (1999),4.231452
2,3,Raiders of the Lost Ark (Indiana Jones and the...,4.216397
3,4,"Shawshank Redemption, The (1994)",4.213172
4,5,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",4.178876
5,6,Star Wars: Episode VII - The Force Awakens (2015),4.159381
6,7,"Wolf of Wall Street, The (2013)",4.156911
7,8,"Imaginarium of Doctor Parnassus, The (2009)",4.141056
8,9,"Prestige, The (2006)",4.118145
9,10,Gone Girl (2014),4.113924


In [ ]:
pass